In [10]:
import pandas as pd
import json, os, requests, logging,  urllib.parse
import plotly.express as px

pl_tag = "#2Q0U9PJLP"
api_key = os.getenv('6498b255-3996-4b71-ab5c-9145dff6d569')
logging.basicConfig(filename='missing_data.log', level=logging.INFO)
def fetch_battle_log():
    url = f"https://api.brawlstars.com/v1/players/{urllib.parse.quote(pl_tag)}/battlelog"
    response = requests.get(url, headers={"Authorization": f"Bearer {api_key}"})
    if response.status_code != 200:
        print(f"Failed to retrieve data for {pl_tag}: {response.status_code}")
        logging.error(f"Failed to retrieve data for {pl_tag}: {response.status_code}")
        return []
    return response.json().get('items', [])
    # print(response)
fetch_battle_log()


ERROR:root:Failed to retrieve data for #2Q0U9PJLP: 403


Failed to retrieve data for #2Q0U9PJLP: 403


[]

In [2]:
import pandas as pd
import json, os, requests, logging,  urllib.parse
import plotly.express as px

pl_tag = "#2Q0U9PJLP"
with open('brawl.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
df = pd.DataFrame(data["items"])
print(df.info())

battles_df = pd.json_normalize(df['battle'])
# print(battles_df.info())
team_count = []
teams_df = pd.json_normalize(battles_df["teams"])
for x in teams_df:
    team_count.append(teams_df[x])



teams_df =  pd.concat(team_count)
teams_df = pd.json_normalize(teams_df.explode('brawlers'))
pl_brawlers = teams_df[teams_df["tag"] == pl_tag]
teamates_df = teams_df[teams_df["tag"] != pl_tag]
# print(teamates_df)
results = battles_df["result"].combine_first(battles_df["rank"])



# teamsdd = pd.json_normalize(teams_df["tag"])

brawler_trophy = teamates_df.groupby("brawler.name")
# print(brawler_trophy["brawler.trophies"].mean())
# print(brawler_trophy["brawler.name"].first())

# print(battles_df["rank"].fillna(0).info(), battles_df["mode"].info())
# print(battles_df["rank"])
print(teamates_df["brawler.power"])






<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   battleTime  25 non-null     object
 1   event       25 non-null     object
 2   battle      25 non-null     object
dtypes: object(3)
memory usage: 728.0+ bytes
None
0       9.0
1       9.0
3      11.0
5      11.0
6       9.0
       ... 
226     NaN
227     NaN
228     NaN
229     NaN
230     NaN
Name: brawler.power, Length: 206, dtype: float64


In [3]:
battles_df['victory'] = battles_df['result'].apply(lambda x: 1 if x == "victory" else 0)
battles_df['defeat'] = battles_df['result'].apply(lambda x: 1 if x != "victory" else 0)

# Agregace dat pro každý mód
agg_df = battles_df.groupby('mode').sum()[['victory', 'defeat']].reset_index()

# Tavení DataFrame pro použití ve sloupcovém grafu
melted_df = agg_df.melt(id_vars='mode', value_vars=['victory', 'defeat'], var_name='result_type', value_name='count')


fig = px.bar(melted_df, x='mode', y='count', color='result_type', barmode='group',
             labels={'mode': 'Mode', 'count': 'Count', 'result_type': 'Result'},
             title='Wins and Losses by Mode')



fig.show()

In [6]:

w_counter = battles_df["result"].apply(lambda x:1 if x == "victory" else 0).sum()
l_counter = len(battles_df["result"]) - w_counter
fig = px.pie(values=[w_counter,l_counter], names=["wins","losses"], title="winsxlosses ratio")
fig.show()

In [56]:
fig = px.treemap(path=[battles_df["mode"],pl_brawlers["brawler.name"],results],title="used brawler by player vs wins/looses")

fig.show()

In [74]:
fig = px.treemap(path=[teamates_df["brawler.name"].dropna(),teamates_df["brawler.power"].dropna(),teamates_df["name"].dropna()],title="most used brawlers")
fig.show()

In [9]:
fig = px.line(x=brawler_trophy["brawler.name"].first(),y=brawler_trophy["brawler.trophies"].mean(),title="mean trophies on teamates brawlers")

fig.show()